# Load Libraries

In [ ]:
import os,gc
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import warnings

# 禁用所有警告
warnings.filterwarnings('ignore')

# Load Train Data

In [ ]:
PATH = '/kaggle/input/hms-harmful-brain-activity-classification/'
df = pd.read_csv(PATH + 'train.csv')
TARGETS = df.columns[-6:]
print('Train shape:', df.shape )
print('Targets', list(TARGETS))
df.head()

# Create Non-Overlapping Eeg Id Train Data

In [ ]:
train = df.groupby('eeg_id')[['spectrogram_id','spectrogram_label_offset_seconds']].agg(
    {'spectrogram_id':'first','spectrogram_label_offset_seconds':'min'})
train.columns = ['spectrogram_id','min']

tmp = df.groupby('eeg_id')[['spectrogram_id','spectrogram_label_offset_seconds']].agg(
    {'spectrogram_label_offset_seconds':'max'})
train['max'] = tmp

tmp = df.groupby('eeg_id')[['patient_id']].agg('first')
train['patient_id'] = tmp

tmp = df.groupby('eeg_id')[TARGETS].agg('sum')
for t in TARGETS:
    train[t] = tmp[t].values
    
y_data = train[TARGETS].values
y_data = y_data / y_data.sum(axis=1,keepdims=True)
train[TARGETS] = y_data

tmp = df.groupby('eeg_id')[['expert_consensus']].agg('first')
train['target'] = tmp

train = train.reset_index()
print('Train non-overlapp eeg_id shape:', train.shape )
train.head(5)

In [ ]:
EEG_PATH = PATH + 'train_eegs/'

row = train.iloc[1]
eeg_id = row['eeg_id']

# 读取 sample EEG 数据
eeg_file_path = os.path.join(EEG_PATH, f'{eeg_id}.parquet')
sample_eeg_data = pd.read_parquet(eeg_file_path)
sample_eeg_data = sample_eeg_data.iloc[:, :-1]
sample_eeg_data = sample_eeg_data.iloc[:, [col for col in range(sample_eeg_data.shape[1]) if col not in [8, 9, 10]]]
# 计算起始时间点
start_time_point = int((sample_eeg_data.shape[0] - 10_000) // 2)

# 获取中间50秒的时间点数据
eeg_slice = sample_eeg_data.iloc[start_time_point : start_time_point + 10_000, :]

eeg_slice

In [ ]:
FLAG = False

In [ ]:
if FLAG:
    fs = 200
    EEG_PATH = PATH + 'train_eegs/'

    # 初始化一个数组来保存所有处理后的 EEG 数据
    all_eeg_data = np.zeros((len(train), 10000, 19))

    # 遍历每个 eeg_id
    for row_idx in range(len(train)):
        row = train.iloc[row_idx]
        eeg_id = row['eeg_id']

        # 读取 EEG 数据
        eeg_file_path = os.path.join(EEG_PATH, f'{eeg_id}.parquet')
        eeg_data = pd.read_parquet(eeg_file_path)
        eeg_data = eeg_data.iloc[:, :-1]
        #eeg_data = eeg_data.iloc[:, [col for col in range(eeg_data.shape[1]) if col not in [8, 9, 10]]]
        # 计算起始时间点
        start_time_point = int((eeg_data.shape[0] - 10_000) // 2)

        # 获取中间50秒的时间点数据
        eeg_slice = eeg_data.iloc[start_time_point : start_time_point + 10_000, :]

        # 更新 all_eeg_data
        all_eeg_data[row_idx, :, :] = eeg_slice

    # 输出数组的形状
    print("Shape of all_eeg_data:", all_eeg_data.shape)
else:
    print("FLAG is set to False. The code below is not executed.")

# Feature Extraction

In [ ]:
import scipy.stats

def extract_eeg_features(eeg_data):
    """
    Extract features from EEG data, including LL Spec, LP Spec, RP Spec, RL Spec
    
    Parameters:
    - eeg_data: EEG data with shape (number of samples, time points, number of electrodes)
    
    Returns:
    - Feature matrix with shape (number of samples, number of features)
    """
    # Initialize the feature matrix
    num_samples, num_time_points, num_electrodes = eeg_data.shape
    num_features = 8 * num_electrodes  # Mean, standard deviation, minimum, maximum, skewness, kurtosis, energy, entropy for each electrode + 4 global features
    features = np.zeros((num_samples, num_features))

    # Extract features
    for sample_idx in range(num_samples):
        for electrode_idx in range(num_electrodes):
            electrode_data = eeg_data[sample_idx, :, electrode_idx]
            feature_idx = electrode_idx * 8

            # Mean
            features[sample_idx, feature_idx] = np.mean(electrode_data)
            # Standard deviation
            features[sample_idx, feature_idx + 1] = np.std(electrode_data)
            # Minimum
            features[sample_idx, feature_idx + 2] = np.min(electrode_data)
            # Maximum
            features[sample_idx, feature_idx + 3] = np.max(electrode_data)
            # Skewness
            features[sample_idx, feature_idx + 4] = scipy.stats.skew(electrode_data)
            # Kurtosis
            features[sample_idx, feature_idx + 5] = scipy.stats.kurtosis(electrode_data)
            # Energy
            features[sample_idx, feature_idx + 6] = np.sum(electrode_data**2) / len(electrode_data)
            # Entropy
            features[sample_idx, feature_idx + 7] = scipy.stats.entropy(np.abs(electrode_data))
        """
        # Additional features: LL Spec, LP Spec, RP Spec, RL Spec
        ll_spec = (np.abs(np.fft.fft(eeg_data[sample_idx, :, 0] - eeg_data[sample_idx, :, 4]))**2 +
                   np.abs(np.fft.fft(eeg_data[sample_idx, :, 4] - eeg_data[sample_idx, :, 5]))**2 +
                   np.abs(np.fft.fft(eeg_data[sample_idx, :, 5] - eeg_data[sample_idx, :, 6]))**2 +
                   np.abs(np.fft.fft(eeg_data[sample_idx, :, 6] - eeg_data[sample_idx, :, 7]))**2) / 4
        features[sample_idx, -4] = np.mean(ll_spec)

        lp_spec = (np.abs(np.fft.fft(eeg_data[sample_idx, :, 0] - eeg_data[sample_idx, :, 1]))**2 +
                   np.abs(np.fft.fft(eeg_data[sample_idx, :, 1] - eeg_data[sample_idx, :, 2]))**2 +
                   np.abs(np.fft.fft(eeg_data[sample_idx, :, 2] - eeg_data[sample_idx, :, 3]))**2 +
                   np.abs(np.fft.fft(eeg_data[sample_idx, :, 3] - eeg_data[sample_idx, :, 7]))**2) / 4
        features[sample_idx, -3] = np.mean(lp_spec)

        rp_spec = (np.abs(np.fft.fft(eeg_data[sample_idx, :, 8] - eeg_data[sample_idx, :, 9]))**2 +
                   np.abs(np.fft.fft(eeg_data[sample_idx, :, 9] - eeg_data[sample_idx, :, 10]))**2 +
                   np.abs(np.fft.fft(eeg_data[sample_idx, :, 10] - eeg_data[sample_idx, :, 11]))**2 +
                   np.abs(np.fft.fft(eeg_data[sample_idx, :, 11] - eeg_data[sample_idx, :, 15]))**2) / 4
        features[sample_idx, -2] = np.mean(rp_spec)

        rl_spec = (np.abs(np.fft.fft(eeg_data[sample_idx, :, 8] - eeg_data[sample_idx, :, 12]))**2 +
                   np.abs(np.fft.fft(eeg_data[sample_idx, :, 12] - eeg_data[sample_idx, :, 13]))**2 +
                   np.abs(np.fft.fft(eeg_data[sample_idx, :, 13] - eeg_data[sample_idx, :, 14]))**2 +
                   np.abs(np.fft.fft(eeg_data[sample_idx, :, 14] - eeg_data[sample_idx, :, 15]))**2) / 4
        features[sample_idx, -1] = np.mean(rl_spec)
        """
    return features

In [ ]:
import numpy as np
import scipy

from scipy.stats import skew, kurtosis
from scipy.stats.mstats import moment
from sklearn.preprocessing import StandardScaler

if FLAG:

    # 使用函数提取特征
    extracted_features = extract_eeg_features(all_eeg_data)

    # 输出特征矩阵的形状
    print("Shape of extracted features:", extracted_features.shape)
else:
    print("FLAG is set to False. The code below is not executed.")

In [ ]:
if FLAG:
    # 计算每列的平均值
    mean_values = np.nanmean(extracted_features, axis=0)

    # 找到 NaN 值的位置
    nan_indices = np.isnan(extracted_features)

    # 用平均值替代 NaN 值
    extracted_features[nan_indices] = np.take(mean_values, nan_indices.nonzero()[1])

    # 输出替代 NaN 后的特征形状
    print("Shape of eeg_features after replacing NaN values:", extracted_features.shape)
    
    # 保存提取的特征到文件
    save_path = '/kaggle/working/extracted_eeg_features.npy'
    np.save(save_path, extracted_features)
    print(f"Extracted EEG features saved at: {save_path}")
else:
    # 当FLAG为False时，加载之前保存的提取好的EEG特征
    extracted_features = np.load('/kaggle/input/8-basic-feaatures-with-eeg/extracted_eeg_features (1).npy')

    # 输出数组的形状
    print("Shape of extracted features:", extracted_features.shape)

# Label

In [ ]:
train_copy = train.copy()
ycol = [c for c in train_copy.columns if c in ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']]
cd = {'Seizure':'seizure_vote', 'GPD':'gpd_vote', 'LRDA':'lrda_vote', 'Other':'other_vote', 'GRDA':'grda_vote', 'LPD':'lpd_vote'}
train_copy['target'] = train_copy['target'].map(cd)
for i in range(len(train_copy)):
    c = train_copy['target'][i]
    train_copy[c][i] = train_copy[c][i]+10 #adding weight to expert consensus

ysum = train_copy[ycol].sum(axis=1) 
for c in ycol:
    train_copy[c] = (train_copy[c] / ysum).astype(np.float64)

In [ ]:
label = train_copy[ycol]
label.shape

# Model

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

# 假设已有 extracted_features 和 label 数据

X = extracted_features
y_labels = np.argmax(label.values, axis=1)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y_labels, test_size=0.2, random_state=42)

# 初始化 XGBoost 分类器，并指定 tree_method 为 "gpu_hist"
xgb_clf = XGBClassifier(n_estimators=1000,
                        max_depth=6,
                        learning_rate=0.1,
                        objective='multi:softmax',
                        num_class=len(np.unique(y_labels)),
                        early_stopping_rounds=100,
                        verbose=100,
                        tree_method='gpu_hist')  # 指定使用 GPU

# 训练 XGBoost 模型
xgb_clf.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=100)

# 获取模型的准确率
acc_xgb = accuracy_score(y_test, xgb_clf.predict(X_test))

# 预测结果
y_pred_xgb = xgb_clf.predict(X_test)

# 输出模型的准确率
accuracy = accuracy_score(y_test, y_pred_xgb)
print(f"Accuracy: {accuracy}")

# Infer Test and Create Submission CSV

In [ ]:
test = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/test.csv')
print('Test shape',test.shape)
test.head()

In [ ]:
# FEATURE ENGINEER TEST
PATH2 = '/kaggle/input/hms-harmful-brain-activity-classification/test_eegs/'
data_test = np.zeros((len(test), extracted_features.shape[1]))
test_eeg = np.zeros((len(test), 10000, 19))

for k in range(len(test)):
    row = test.iloc[k]
    s = int(row.eeg_id)
    eeg_test = pd.read_parquet(f'{PATH2}{s}.parquet')
    eeg_test = eeg_test.iloc[:, :-1]
    #eeg_test = eeg_test.iloc[:, [col for col in range(eeg_test.shape[1]) if col not in [8, 9, 10]]]
    # 计算起始时间点
    start_time_point = int((eeg_test.shape[0] - 10_000) // 2)
    # 获取中间50秒的时间点数据
    eeg_test_slice = eeg_test.iloc[start_time_point : start_time_point + 10_000, :]
    
    test_eeg[k, :, :] = eeg_test_slice

    # 使用之前定义的 extract_eeg_features 函数进行特征提取
    features_test = extract_eeg_features(test_eeg)
    print("Shape of features_test:", features_test.shape)
    # 将提取的特征放入 data_test 中
    data_test = features_test

# 输出 data_test 的形状
print("Shape of data_test:", data_test.shape)

In [ ]:
# 使用训练好的模型进行预测
predictions_proba = xgb_clf.predict_proba(data_test)

# 类别的名称
class_names = ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']

# 创建一个DataFrame来存储概率值
probabilities_df = pd.DataFrame(predictions_proba, columns=class_names)

# 显示DataFrame
print(probabilities_df.head(1))

In [ ]:
sub = pd.DataFrame({'eeg_id':test.eeg_id.values})
sub[TARGETS] = probabilities_df
sub.to_csv('submission.csv',index=False)
print('Submissionn shape',sub.shape)
sub.head()

In [ ]:
# SANITY CHECK TO CONFIRM PREDICTIONS SUM TO ONE
sub.iloc[:,-6:].sum(axis=1)